This dataset is used to create and prepare the neighborhods dataset for creating the neighborhoods in the model. The CBS Wijk- en Buurtkaart dataset is used as a source.

In [1]:
# import packages

import numpy as np 
import pandas as pd  # provides interface for interacting with tabular data
import geopandas as gpd  # combines the capabilities of pandas and shapely for geospatial operations
from shapely.geometry import Point, Polygon, MultiPolygon  # for manipulating text data into geospatial shapes
from shapely import wkt  # stands for "well known text," allows for interchange across GIS programs
import rtree  # supports geospatial join

# import seaborn as sns
import matplotlib. pyplot as plt # for plotting graphs
import matplotlib.mlab as mlab # 
import matplotlib
plt.style.use('ggplot') # basic, but functional, plotstyle,
from matplotlib.pyplot import figure
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

In [2]:
# read files

neighborhoods_input = gpd.read_file("/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Input/Buurt/buurt_2017_v3.shp")
#labels_input = pd.read_csv('/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/postcode_ins_by_v1.csv')
labels_input = pd.read_csv('/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/postcode_ins_by_woningcat_v1.csv')

In [ ]:
# exclude unnecessary columns

neighborhoods_1 = neighborhoods_input[['BU_CODE','BU_NAAM','GM_CODE','GM_NAAM','WATER','POSTCODE','AANTAL_HH','AANT_INW','STED','G_GAS_TOT', 'P_STADVERW', 'WOZ','P_HUURCORP','P_WONV2000', 'geometry']]

In [ ]:
# missing values output all 

for col in neighborhoods_1.columns: # iterate over columns
    p_missing = np.mean(neighborhoods_1[col].isnull()) # returns how many values in columns are NA on average
    print('{} - {}%'.format(col, round(p_missing*100))) # prints in readable format

In [ ]:
# negative values output

for col in neighborhoods_1.columns: # iterate over columns
    p_99999999 = np.mean(neighborhoods_1[col] == -99999999) # returns how many values in columns are NA on average
    print('{} - {}%'.format(col, round(p_99999999*100))) # prints in readable format

In [ ]:
# boxplotting the variables of interest

fig, axs = plt.subplots(1,7)
fig.suptitle('Before cleaning and preparation')

axs[0].boxplot(neighborhoods_1.AANTAL_HH)
axs[0].set_title('AANTAL_HH')

axs[1].boxplot(neighborhoods_1.STED)
axs[1].set_title('STED')

axs[2].boxplot(neighborhoods_1.G_GAS_TOT)
axs[2].set_title('G_GAS_TOT')

axs[3].boxplot(neighborhoods_1.P_HUURCORP)
axs[3].set_title('P_HUURCORP')

axs[4].boxplot(neighborhoods_1.P_STADVERW)
axs[4].set_title('P_STADVERW')

axs[5].boxplot(neighborhoods_1.P_WONV2000)
axs[5].set_title('P_WONV2000')

axs[6].boxplot(neighborhoods_1.WOZ)
axs[6].set_title('WOZ')

In [ ]:
neighborhoods_1.P_STADVERW[neighborhoods_1.BU_NAAM == "Nederlangbroek West- en Oostzijde"]

In [ ]:
# remove non-existing neighborhoods that are water, fictional or dont have any households (0 households)
neighborhoods_2 = neighborhoods_1[neighborhoods_1.BU_NAAM != 'None']
neighborhoods_3 = neighborhoods_2[neighborhoods_2.WATER == 'NEE']
neighborhoods_3 = neighborhoods_3[neighborhoods_3.AANTAL_HH > 0]

In [ ]:
# boxplotting the variables of interest

fig, axs = plt.subplots(1,7)
fig.suptitle('Before cleaning and preparation, WATER neighborhoods removed')

axs[0].boxplot(neighborhoods_3.AANTAL_HH)
axs[0].set_title('AANTAL_HH')

axs[1].boxplot(neighborhoods_3.STED)
axs[1].set_title('STED')

axs[2].boxplot(neighborhoods_3.G_GAS_TOT)
axs[2].set_title('G_GAS_TOT')

axs[3].boxplot(neighborhoods_3.P_HUURCORP)
axs[3].set_title('P_HUURCORP')

axs[4].boxplot(neighborhoods_3.P_STADVERW)
axs[4].set_title('P_STADVERW')

axs[5].boxplot(neighborhoods_3.P_WONV2000)
axs[5].set_title('P_WONV2000')

axs[6].boxplot(neighborhoods_3.WOZ)
axs[6].set_title('WOZ')

In [ ]:
neighborhoods_3_test = neighborhoods_3[neighborhoods_3.GM_NAAM == "Rotterdam"]
#neighborhoods_3_test
#neighborhoods_3_test.P_STADVERW.loc[7010]

In [ ]:
# adjust non-sensible negative values to averages of neighborhoods

# first add a neighborhoods column to the dataframe
neighborhoods_3['NEIGHBORS'] = None

# iterate over index and add list of neighbors to all neighborhoods
for index, neighborhood in neighborhoods_3.iterrows():
    try:
        neighbors = neighborhoods_3[~neighborhoods_3.geometry.disjoint(neighborhood.geometry)].BU_NAAM.tolist()
        neighbors = [name for name in neighbors if neighborhood.BU_NAAM != name ]
        neighborhoods_3.at[index, "NEIGHBORS"] = neighbors
    except: # due to errors with a few polygons this exception is built in --> instead of neighbors, all neighborhoods of municipality
        gm_naam = neighborhoods_3.GM_NAAM.loc[index]
        neighbors = neighborhoods_3[neighborhoods_3.GM_NAAM == gm_naam].BU_NAAM.tolist()
        neighbors = [name for name in neighbors if neighborhood.BU_NAAM != name ]
        neighborhoods_3.at[index, "NEIGHBORS"] = neighbors





In [ ]:
neighborhoods_3[neighborhoods_3.NEIGHBORS.isnull()]

In [ ]:
for col in neighborhoods_3.columns: # iterate over columns
    p_missing = np.mean(neighborhoods_3[col].isnull()) # returns how many values in columns are NA on average
    print('{} - {}%'.format(col, round(p_missing*100))) # prints in readable format

In [ ]:
# function that reports the average of a given variable of all its neighbors (given a neighbors column exists)if it is a missing value

def avg_neighbors(row, var, df):
    if df[var].loc[row] < 0:
        # create list with my neighbor names
        neighbor_list = df.NEIGHBORS.loc[row]
        
        # calculate mean of desired variable of neighborhoods
        mean = df[var][(df.BU_NAAM.isin(neighbor_list)) & (df[var] >= 0)].mean()
        
        if mean >= 0:
            return mean
        else:
            return 0

In [ ]:
neighborhoods_3_test = neighborhoods_3[neighborhoods_3.GM_NAAM == "Rotterdam"]
neighborhoods_3_test

In [ ]:
# adjust non-sensible negative values to averages of neighborhoods

for index in neighborhoods_3_test.index:
    if neighborhoods_3_test['STED'].loc[index] < 0:
        neighborhoods_3_test['STED'].loc[index] = avg_neighbors(index, 'STED', neighborhoods_3_test)
    if neighborhoods_3_test['G_GAS_TOT'].loc[index] < 0:
        neighborhoods_3_test['G_GAS_TOT'].loc[index] = avg_neighbors(index, 'G_GAS_TOT', neighborhoods_3_test)
    if neighborhoods_3_test['P_HUURCORP'].loc[index] < 0:
        neighborhoods_3_test['P_HUURCORP'].loc[index] = avg_neighbors(index, 'P_HUURCORP', neighborhoods_3_test)
    if neighborhoods_3_test['P_STADVERW'].loc[index] < 0:
        neighborhoods_3_test['P_STADVERW'].loc[index] = 0
    if neighborhoods_3_test['P_WONV2000'].loc[index] < 0:
        neighborhoods_3_test['P_WONV2000'].loc[index] = avg_neighbors(index, 'P_WONV2000', neighborhoods_3_test)
    if neighborhoods_3_test['WOZ'].loc[index] < 0:
        neighborhoods_3_test['WOZ'].loc[index] = avg_neighbors(index, 'WOZ', neighborhoods_3_test)
    
#neighborhoods_3_test.sample(80)

In [ ]:
# adjust non-sensible negative values to averages of neighborhoods

for index in neighborhoods_3.index:
    if neighborhoods_3['STED'].loc[index] < 0:
        neighborhoods_3['STED'].loc[index] = avg_neighbors(index, 'STED', neighborhoods_3)
    if neighborhoods_3['G_GAS_TOT'].loc[index] < 0:
        neighborhoods_3['G_GAS_TOT'].loc[index] = avg_neighbors(index, 'G_GAS_TOT', neighborhoods_3)
    if neighborhoods_3['P_HUURCORP'].loc[index] < 0:
        neighborhoods_3['P_HUURCORP'].loc[index] = avg_neighbors(index, 'P_HUURCORP', neighborhoods_3)
    if neighborhoods_3['P_STADVERW'].loc[index] < 0:
        neighborhoods_3['P_STADVERW'].loc[index] = avg_neighbors(index, 'P_STADVERW', neighborhoods_3)
    if neighborhoods_3['P_WONV2000'].loc[index] < 0:
        neighborhoods_3['P_WONV2000'].loc[index] = avg_neighbors(index, 'P_WONV2000', neighborhoods_3)
    if neighborhoods_3['WOZ'].loc[index] < 0:
        neighborhoods_3['WOZ'].loc[index] = avg_neighbors(index, 'WOZ', neighborhoods_3)
    
# neighborhoods_3.sample(90)

In [ ]:
# missing values output all 

for col in neighborhoods_3.columns: # iterate over columns
    p_missing = np.mean(neighborhoods_3[col].isnull()) # returns how many values in columns are NA on average
    print('{} - {}%'.format(col, round(p_missing*100))) # prints in readable format

In [ ]:
# negative values output

for col in neighborhoods_3.columns: # iterate over columns
    p_99999999 = np.mean(neighborhoods_3[col] == -99999999) # returns how many values in columns are NA on average
    print('{} - {}%'.format(col, round(p_99999999*100))) # prints in readable format

In [ ]:
neighborhoods_3.boxplot()

In [ ]:
fig, axs = plt.subplots(1)
fig.suptitle('After cleaning and preparation')

axs.boxplot(neighborhoods_3.STED)
axs.set_title('AANTAL_HH')

In [ ]:
# boxplotting the variables of interest after cleaning and preparation

fig, axs = plt.subplots(1,7)
fig.suptitle('After cleaning and preparation')

axs[0].boxplot(neighborhoods_3.AANTAL_HH)
axs[0].set_title('AANTAL_HH')

axs[1].boxplot(neighborhoods_3.STED)
axs[1].set_title('STED')

axs[2].boxplot(neighborhoods_3.G_GAS_TOT)
axs[2].set_title('G_GAS_TOT')

axs[3].boxplot(neighborhoods_3.P_HUURCORP)
axs[3].set_title('P_HUURCORP')

axs[4].boxplot(neighborhoods_3.P_STADVERW)
axs[4].set_title('P_STADVERW')

axs[5].boxplot(neighborhoods_3.P_WONV2000)
axs[5].set_title('P_WONV2000')

axs[6].boxplot(neighborhoods_3.WOZ)
axs[6].set_title('WOZ')

In [ ]:
# clean data and fill non-sensible (below zero) variables with averages of neighbors

# remove unnessesary columns

# remove entries that are classified as WATER

# remove entries that have no households

# put averages 

In [ ]:
# transform data

# adding the energy labels datast to the neighborhoods dataframe
neighborhoods_7 = neighborhoods_3.copy()
neighborhoods_7['POSTCODE'] = neighborhoods_3['POSTCODE'].apply(pd.to_numeric)
neighborhoods_8 = neighborhoods_7.merge(labels_input, how = 'left', left_on='POSTCODE', right_on = 'POSTCODE_DIGITS')
# remove entries that slipped through that have no such values
neighborhoods_9 = neighborhoods_8[(neighborhoods_8['STED'] > 0) & (neighborhoods_8['LABELCAT'] > 0)]

In [ ]:
# transforming the projection system to WGS84 to be readable for Netlogo
neighborhoods_9_WGS84 = neighborhoods_9.to_crs("EPSG:32632")
neighborhoods_9_WGS84.crs

In [ ]:
neighborhoods_10_WGS84 = neighborhoods_9_WGS84.drop('NEIGHBORS', axis=1)

In [ ]:
neighborhoods_10_WGS84.WOZ.isnull().value_counts()

In [ ]:
neighborhoods_10_WGS84.P_STADVERW[neighborhoods_10_WGS84.BU_NAAM == "Nederlangbroek West- en Oostzijde"]

In [ ]:
# output data
neighborhoods_10_WGS84.to_file("/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/Buurt/30_07/neighborhoods.shp")

In [9]:
# output data descriptives
nh_descriptives = gpd.read_file("/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/Buurt/30_07/neighborhoods.shp")

In [20]:
df_latex1 = nh_descriptives[['AANTAL_HH', 'STED', 'P_STADVERW', 'WOZ', 'P_HUURCORP', 'P_WONV2000']].describe()

In [21]:
print(df_latex1.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &     AANTAL\_HH &          STED &    P\_STADVERW &           WOZ &    P\_HUURCORP &    P\_WONV2000 \\
\midrule
count &  12603.000000 &  12603.000000 &  12603.000000 &  12603.000000 &  12603.000000 &  12603.000000 \\
mean  &    616.886852 &      3.593589 &     13.155662 &    250.613512 &     18.242060 &     15.349301 \\
std   &    831.307088 &      1.487139 &     17.569534 &    109.757676 &     20.657545 &     21.328644 \\
min   &      5.000000 &      1.000000 &      0.000000 &      0.000000 &      0.000000 &      0.000000 \\
25\%   &     80.000000 &      2.000000 &      0.000000 &    182.000000 &      0.000000 &      3.000000 \\
50\%   &    320.000000 &      4.000000 &      8.000000 &    234.000000 &     13.000000 &      9.000000 \\
75\%   &    840.000000 &      5.000000 &     18.613103 &    298.000000 &     29.000000 &     17.000000 \\
max   &  13950.000000 &      5.000000 &    100.000000 &   1633.000000 &    100.000000 &    100.000000 \\
\bottomr

In [22]:
df_latex2 = nh_descriptives[['LABELCAT', 'FLAT_APP', 'VRIJ', 'RIJHOEK', 'RIJTUSSEN', 'TWEE_KAP']].describe()

In [23]:
print(df_latex2.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &      LABELCAT &      FLAT\_APP &          VRIJ &       RIJHOEK &     RIJTUSSEN &      TWEE\_KAP \\
\midrule
count &  12603.000000 &  12603.000000 &  12603.000000 &  12603.000000 &  12603.000000 &  12603.000000 \\
mean  &      3.143605 &      0.240946 &      0.270399 &      0.114703 &      0.219623 &      0.148651 \\
std   &      0.952924 &      0.231412 &      0.243845 &      0.060932 &      0.144292 &      0.103272 \\
min   &      0.048780 &      0.000000 &      0.000000 &      0.000000 &      0.000000 &      0.000000 \\
25\%   &      2.580533 &      0.078404 &      0.066082 &      0.071375 &      0.099008 &      0.062913 \\
50\%   &      3.174914 &      0.163522 &      0.205607 &      0.121745 &      0.208448 &      0.142857 \\
75\%   &      3.668157 &      0.324902 &      0.421887 &      0.160648 &      0.329387 &      0.221936 \\
max   &      6.000000 &      1.000000 &      1.000000 &      0.500000 &      0.800407 &      0.823529 \\
\bottomrul

In [24]:
insulation_input = pd.read_csv('/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Input/Voorlopige-labels-december-2019_FAST_v1.csv', low_memory = False)

In [26]:
insulation_input

,POSTCODE_WONING,HUISNUMMER_WONING,HUISNUMMER_TOEV_WONING,BOUWJAAR_WONING,WONING_TYPE,VOORL_BEREKEND
0,9901KB,16.0,NaN,2002.0,Flat/appartement,B
1,9901KB,20.0,NaN,2002.0,Flat/appartement,B
2,9901AD,9.0,NaN,1800.0,Niet wonen,NaN
3,9901AD,13.0,NaN,1920.0,Niet wonen,NaN
4,9901AD,15.0,NaN,1925.0,Vrijstaande woning,G
...,...,...,...,...,...,...
9070192,1601EE,31.0,NaN,2016.0,Rijwoning hoek,A
9070193,1601AC,65.0,A,1659.0,Flat/appartement,G
9070194,1602CS,20.0,NaN,2017.0,Vrijstaande woning,A
9070195,1602CS,24.0,NaN,2017.0,Vrijstaande woning,A


In [27]:
df_latex3 = insulation_input[['POSTCODE_WONING', 'WONING_TYPE', 'VOORL_BEREKEND']].describe()

In [29]:
print(df_latex3.to_latex())

\begin{tabular}{llll}
\toprule
{} & POSTCODE\_WONING &       WONING\_TYPE & VOORL\_BEREKEND \\
\midrule
count  &         9070197 &           9070197 &        8317556 \\
unique &          459632 &                 9 &              8 \\
top    &          1033SC &  Flat/appartement &              C \\
freq   &            1107 &           2857292 &        2283250 \\
\bottomrule
\end{tabular}

